In [1]:
import torch
import torch.nn as nn

In [2]:
class AlexNet1D(nn.Module): # Without MaxPooling layers
    def __init__(self, in_channels, num_classes):
        super(AlexNet1D, self).__init__()
        
        self.conv1 = nn.Conv1d(in_channels, out_channels=4096, kernel_size=11, stride=4, bias=False)
        self.conv2 = nn.Conv1d(in_channels=4096, out_channels=2048, kernel_size=5, stride=2, padding=2, bias=False)
        self.conv3 = nn.Conv1d(in_channels=2048, out_channels=1024, kernel_size=3, padding=1, bias=True)
        self.conv4 = nn.Conv1d(in_channels=1024, out_channels=512, kernel_size=3, padding=1, bias=True)
        self.conv5 = nn.Conv1d(in_channels=512, out_channels=256, kernel_size=3, padding=1, bias=False)
        
        self.fc1 = nn.Linear(in_features=512, out_features=4096)
        self.fc2 = nn.Linear(in_features=4096, out_features=4096)
        self.fc3 = nn.Linear(in_features=4096, out_features=num_classes)
        
        self.drop = nn.Dropout(p=0.5)
        self.relu = nn.ReLU(inplace=True)
        self.bn1 = nn.BatchNorm1d(num_features=4096)
        self.bn2 = nn.BatchNorm1d(num_features=2048)
        self.bn3 = nn.BatchNorm1d(num_features=1024)
        self.bn4 = nn.BatchNorm1d(num_features=512)
        self.bn5 = nn.BatchNorm1d(num_features=256)
        self.max_pool = nn.MaxPool1d(kernel_size=3, stride=2)
    
    def forward(self, x):
        b_size = x.size(0)
        x = self.relu(self.bn1(self.conv1(x)))
        # x = self.max_pool(x)
        x = self.relu(self.bn2(self.conv2(x)))
        # x = self.max_pool(x)
        
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.relu(self.bn4(self.conv4(x)))
        x = self.relu(self.bn5(self.conv5(x)))
        # x = self.max_pool(x)
        
        x = x.reshape(b_size, 6, -1)
        
        x = self.relu(self.drop(self.fc1(x)))
        x = self.relu(self.drop(self.fc2(x)))
        x = self.relu(self.drop(self.fc3(x)))       
        return x

In [3]:
model = AlexNet1D(in_channels=6044, num_classes=9)

x = torch.randn((2, 6044, 99))
model(x).shape

torch.Size([2, 6, 9])